# PyTorch Tutorial: Graph Neural Networks (GNNs)

Graphs are everywhere: Social Networks (Facebook), Knowledge Graphs (Google), and Molecule Structures (Drug Discovery). **Graph Neural Networks (GNNs)** are the state-of-the-art for learning on this non-Euclidean data.

## Learning Objectives
- Understand **Graphs**: Nodes, Edges, and Adjacency Matrices.
- Understand **Message Passing**: How nodes talk to neighbors.
- Implement a **GCN (Graph Convolutional Network)** using `torch_geometric`.
- Solve a **Link Prediction** task (Recommendation System).

## 1. Vocabulary First

- **Node (Vertex)**: An entity (e.g., a User, a Product).
- **Edge (Link)**: A connection (e.g., "Friend of", "Bought").
- **Feature Matrix ($X$)**: Attributes of each node (e.g., Age, Location).
- **Adjacency Matrix ($A$)**: A grid showing who is connected to whom.
- **Message Passing**: Aggregating information from neighbors to update a node's embedding.

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

print("Ready for Graphs!")

## 2. Creating a Simple Graph

Let's create a small social network with 3 people.
- Node 0 is friends with Node 1.
- Node 1 is friends with Node 2.

In [ ]:
# Edge Index (COO format): [Source Nodes, Target Nodes]
edge_index = torch.tensor([
    [0, 1, 1, 2],
    [1, 0, 2, 1]
], dtype=torch.long)

# Node Features (e.g., Age, Activity Level) - 2 features per node
x = torch.tensor([
    [-1, 0], # Node 0
    [ 0, 1], # Node 1
    [ 1, 0]  # Node 2
], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
print(data)

## 3. Graph Convolutional Network (GCN)

A GCN layer updates a node's representation by averaging its neighbors' features.

$$ h_v^{(l+1)} = \sigma \left( \sum_{u \in \mathcal{N}(v)} \frac{1}{c_{uv}} W^{(l)} h_u^{(l)} \right) $$

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(2, 16) # Input: 2 features -> Hidden: 16
        self.conv2 = GCNConv(16, 2) # Hidden: 16 -> Output: 2 (Embedding)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return x

model = GCN()
print(model)

## 4. Link Prediction (Recommendation)

To recommend a friend, we check if the dot product of two node embeddings is high.

$$ Score(u, v) = h_u \cdot h_v $$

In [ ]:
# Forward pass to get embeddings
embeddings = model(data)

# Predict link between Node 0 and Node 2 (who are NOT friends yet)
node_0 = embeddings[0]
node_2 = embeddings[2]

score = torch.matmul(node_0, node_2)
prob = torch.sigmoid(score)

print(f"Probability of connection between 0 and 2: {prob.item():.4f}")

## Key Takeaways

1. **Graphs** model relationships.
2. **GNNs** learn embeddings based on graph structure.
3. **Link Prediction** is the basis of modern Recommendation Systems.